### Sample program for VAE denoising (Dense)   
- For train: input: with noise / output: without noise  

#### Import libraries  

In [ ]:
from tensorflow.keras import backend as K
#from keras import backend as K
from tensorflow.keras.layers import Lambda, Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.losses import binary_crossentropy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import  os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True' 

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
np.random.seed(17)
tf.random.set_seed(23)

#### Parameters  

In [ ]:
n_digits = 5
n_train = 4800
n_test = 800

#### Function for Z-sampling between encoder and decoder  

In [ ]:
# reparameterization trick
# instead of sampling from Q(z|X), sample epsilon = N(0,I)
# z = z_mean + sqrt(var) * epsilon
def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.

    # Arguments
        args (tensor): mean and log of variance of Q(z|X)

    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim),
                              mean=0., stddev=1. )
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

#### Function for visualization of data in latent space  

In [ ]:
def plot_results(models, data, batch_size=128):
    """Plots labels and MNIST digits as a function of the 2D latent vector

    # Arguments
        models (tuple): encoder and decoder models
        data (tuple): test data and label
        batch_size (int): prediction batch size
    """

    encoder, decoder = models
    test_data, test_label = data

    # display a 2D manifold of digits
    z_mean1, _, _ = encoder.predict(test_data,
                                    batch_size=batch_size)
    plt.figure(figsize=(9, 7))
    plt.scatter(z_mean1[:, 0], z_mean1[:, 1], c=test_label)
    plt.colorbar()
    plt.xlabel("z1")
    plt.ylabel("z2")
    plt.xlim(-4, 4)
    plt.ylim(-4, 4)
    plt.show()

    # display a 2D manifold of digits
    n = 15
    figure = np.zeros((img_h * n, img_w * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-4, 4, n)
    grid_y = np.linspace(-4, 4, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(img_h, img_w)
            figure[i * img_h: (i + 1) * img_h,
                   j * img_w: (j + 1) * img_w] = digit

    plt.figure(figsize=(7, 7))
    start_range_x = img_w // 2
    end_range_x = (n - 1) * img_w + start_range_x + 1
    pixel_range_x = np.arange(start_range_x, end_range_x, img_w)
    start_range_y = img_h // 2
    end_range_y = (n - 1) * img_h + start_range_y + 1
    pixel_range_y = np.arange(start_range_y, end_range_y, img_h)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range_x, sample_range_x)
    plt.yticks(pixel_range_y, sample_range_y)
    plt.xlabel("z1")
    plt.ylabel("z2")
    plt.imshow(figure, cmap='Greys_r')
    plt.show()

#### Load MNIST  

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

#### Select only 0..(n_train/test_digits-1)  

In [ ]:
x_train = x_train[ y_train < n_digits ]
y_train = y_train[ y_train < n_digits ]
x_test = x_test[ y_test < n_digits ]
y_test = y_test[ y_test < n_digits ]
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

#### Check the number of each label  

In [ ]:
print(pd.Series(y_train).value_counts())
print(pd.Series(y_test).value_counts())

#### Reduce the number of data (for shorter calculation time)  

In [ ]:
x_train = x_train[:n_train]
y_train = y_train[:n_train]
x_test = x_test[:n_test]
y_test = y_test[:n_test]
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

#### Check the number of each label  

In [ ]:
print(pd.Series(y_train).value_counts())
print(pd.Series(y_test).value_counts())

#### Obtain size of image  

In [ ]:
img_h = x_train.shape[1]
img_w = x_train.shape[2]
original_dim = img_h * img_w
print(img_h, img_w, original_dim)

#### Re-scale data (0..1) and flatten  

In [ ]:
x_train = x_train.astype('float32') / 255
x_train = x_train.reshape(-1, original_dim)
print(x_train.shape)
x_test = x_test.astype('float32') / 255
x_test = x_test.reshape(-1, original_dim)
print(x_test.shape)

#### Generate data with noise  

In [ ]:
noise_train = np.random.normal(loc=0.15, scale=0.3,
                               size=x_train.shape)
noise_test = np.random.normal(loc=0.15, scale=0.3,
                              size=x_test.shape)

x_train_noisy = np.clip(x_train+noise_train, 0, 1)
print(x_train_noisy.shape)
x_test_noisy = np.clip(x_test+noise_test, 0, 1)
print(x_test_noisy.shape)

#### See images without noise  

In [ ]:
fig = plt.figure(figsize=(8, 8))  # don't move to other cell
tile_h = 5
tile_w = 5
for i in range(tile_h*tile_w):
    fig.add_subplot(tile_h, tile_w, i+1, xticks=[], yticks=[])
    plt.imshow(x_train[i].reshape((img_h, img_w)), cmap='gray')
    
plt.show()

#### See images with noise  

In [ ]:
fig = plt.figure(figsize=(8, 8))  # don't move to other cell
tile_h = 5
tile_w = 5
for i in range(tile_h*tile_w):
    fig.add_subplot(tile_h, tile_w, i+1, xticks=[], yticks=[])
    plt.imshow(x_train_noisy[i].reshape((img_h, img_w)), cmap='gray')
    
plt.show()

#### Start VAE calculation  

In [ ]:
K.clear_session()

#### Parameters for dense and latent space  

In [ ]:
input_shape = (original_dim, )
intermediate_dim = 512
latent_dim = 2

#### Build VAE model = encoder + decoder

In [ ]:
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(intermediate_dim, activation='relu')(inputs)
x = Dense(intermediate_dim, activation='relu')(x)
x = Dense(intermediate_dim, activation='relu')(x)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
z = Lambda(sampling, name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

In [ ]:
# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim, activation='relu')(latent_inputs)
x = Dense(intermediate_dim, activation='relu')(x)
x = Dense(intermediate_dim, activation='relu')(x)
outputs = Dense(original_dim, activation='sigmoid')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

#### Define the whole model  

In [ ]:
# instantiate VAE model
outputs = decoder(encoder(inputs)[2])  # encoder(input)[2]: z
vae = Model(inputs, outputs, name='vae')

#### Function to calculate loss  

In [ ]:
def vae_loss(y_true, y_pred):
    reconstruction_loss = binary_crossentropy(y_true,
                                              y_pred)
    reconstruction_loss *= original_dim
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1) / 2
    ret = K.mean(reconstruction_loss - kl_loss)
    return ret

#### Set optimization parameters  

In [ ]:
vae.compile(optimizer='adam', loss=vae_loss,
            experimental_run_tf_function=False)
vae.summary()

#### Exec learning (training)  

In [ ]:
batch_size = 32
epochs = 40
val_split = 0.1
input_data = x_train_noisy
teacher_data = x_train
input_test_data = x_test_noisy
true_test_data = x_test
true_test_label = y_test

In [ ]:
%%time

fit_log = vae.fit(input_data, teacher_data,
                  epochs=epochs,
                  batch_size=batch_size,
                  validation_split=val_split)

#### Plot of loss/val_loss vs epochs  

In [ ]:
plt.plot(fit_log.history['loss'], label='train')
plt.plot(fit_log.history['val_loss'], label='valid')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

#### Analysis of the results   

In [ ]:
models = (encoder, decoder)
data = (input_test_data, y_test)

plot_results(models, data,
             batch_size=batch_size)

##### Obtain output for test_data      

In [ ]:
pred = vae.predict(input_test_data)

##### Show sample images  

In [ ]:
n = 10
plt.figure(figsize=(12, 4))
for i in range(n):
    # Show input test image
    ax = plt.subplot(3, n, i+1)
    plt.imshow(input_test_data[i].reshape(img_h, img_w))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
 
    # Show output test image
    ax = plt.subplot(3, n, i+1+n)
    plt.imshow(pred[i].reshape(img_h, img_w))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # Obtain diff between images
    diff_img = true_test_data[i] - pred[i]
    
    # Obtain total value of diff between images  
    diff_score = np.sum(np.abs(true_test_data[i]-pred[i]))
        
    # Show results  
    ax = plt.subplot(3, n, i+1+n*2)
    plt.imshow(diff_img.reshape(img_h, img_w))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False) 
    ax.set_xlabel('score:{:.2f}'.format(diff_score), fontsize=8)    
    
plt.show()

##### Show distribution of scores  

In [ ]:
diff_scores = []
diff_labels = []
for i in range(len(input_test_data)):   
    # Obtain total value of diff between images  
    diff_score = np.sum(np.abs(true_test_data[i]-pred[i]))
    diff_scores.append(diff_score)
    diff_labels.append(true_test_label[i])

diff_scores = np.array(diff_scores)
diff_labels = np.array(diff_labels)

for i in range(n_digits):
    plt.hist(diff_scores[ diff_labels==i ], label=str(i), density=True, alpha=0.2)
plt.xlabel('Scores')
plt.ylabel('Prob')
plt.legend()
plt.show()